## Import library

In [3]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


In [4]:
tf.__version__

'2.2.0'

In [5]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [6]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [5]:

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    print("Data index {}. Delta {:.2f}. Loss {:.2f}".format(index_, delta_r, loss.numpy()))

  0%|          | 0/100 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(losses)

In [ ]:
data = asarray(delta_seq)
save('delta.npy', delta_seq)

## Optimization with list of delta

In [14]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=1.0, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [18]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(1, 10)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    print(delta_r.shape)
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))
 

 11%|█         | 1/9 [04:49<38:39, 289.88s/it]

(672,)
Data index 1. Loss 0.00


 22%|██▏       | 2/9 [04:54<23:50, 204.32s/it]

The performance reach MAE: 0.001. Cancelling the training at step 31
(672,)
Data index 2. Loss 0.00


 33%|███▎      | 3/9 [09:35<22:43, 227.29s/it]

...

(672,)
Data index 7. Loss 0.05


 89%|████████▉ | 8/9 [33:31<04:37, 277.51s/it]

(672,)
Data index 8. Loss 0.02


100%|██████████| 9/9 [38:27<00:00, 256.36s/it]

(672,)
Data index 9. Loss 0.26


In [21]:
np.array(delta_seq).shape

In [22]:
save('list_delta_.npy', delta_seq)






## Optimization for individual treatment effect

In [8]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(100000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 1000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [9]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0. Loss 128.21106
Epoch 1000. Loss 35.25104
Epoch 2000. Loss 26.33487
Epoch 3000. Loss 22.03906
Epoch 4000. Loss 19.51452
Epoch 5000. Loss 17.79730
Epoch 6000. Loss 16.47934
Epoch 7000. Loss 15.40867
Epoch 8000. Loss 14.58724
Epoch 9000. Loss 13.90437
Epoch 10000. Loss 13.29880
Epoch 11000. Loss 12.75287
Epoch 12000. Loss 12.24901
Epoch 13000. Loss 11.83835
Epoch 14000. Loss 11.50295
Epoch 15000. Loss 11.18480
Epoch 16000. Loss 10.90050
Epoch 17000. Loss 10.66358
Epoch 18000. Loss 10.45095
Epoch 19000. Loss 10.25655
Epoch 20000. Loss 10.07695
Epoch 21000. Loss 9.91892
Epoch 22000. Loss 9.77304
Epoch 23000. Loss 9.63611
Epoch 24000. Loss 9.50814
Epoch 25000. Loss 9.38981
Epoch 26000. Loss 9.27811
Epoch 27000. Loss 9.17255
Epoch 28000. Loss 9.07324
Epoch 29000. Loss 8.97934
Epoch 30000. Loss 8.89013
Epoch 31000. Loss 8.80546
Epoch 32000. Loss 8.72501
Epoch 33000. Loss 8.64893
Epoch 34000. Loss 8.57677
Epoch 35000. Loss 8.50806
Epoch 36000. Loss 8.44242
Epoch 37000. Loss 8.37966
Epo

100%|██████████| 1/1 [09:38<00:00, 578.35s/it]

Data index 13. Loss 6.87


In [12]:
save('individual_list_delta_.npy', delta_seq)

## Genetic algorithm

In [15]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"


index_ = 10
data = utils.load_data(PATH_TRAIN, index_)
data_test = utils.load_data(PATH_TEST, index_)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

data['p1'] = model_t.predict_proba(data[cov])[:,1]
data['p0'] = 1 - data['p1']

## Compute counterfactual outcome with no treatment
data_pos = data.copy()
data_pos[treatment] = 1
data['cf1'] = model_y.predict(data_pos[features])

## Compute counterfactual outcome with treatment
data_neg = data.copy()
data_neg[treatment] = 0
data['cf0'] = model_y.predict(data_neg[features])

data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                      (1 - data['p1']))

data['w0'] = data['ips_weight']*data[treatment]
data['w1'] = data['ips_weight']*(1 - data[treatment])

In [28]:
true_effect = data['mu1'] - data['mu0']

### Creating the initial population.

In [176]:
# initial_deltas = []
# initial_delta = np.random.randint(0, high=100, size=[100,data.shape[0]])
# initial_deltas.append(initial_delta)
initial_delta = np.random.rand(1000,data.shape[0])
# initial_deltas.append(initial_delta)

In [177]:
# initial_deltas = np.array(initial_deltas)
# initial_deltas = initial_deltas.reshape(200,672)

### Fitness function


In [178]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def fitness_function(influence, true_effect):
    return 1 / np.mean(utils.abs_ate(influence, true_effect))


In [182]:
individuals = []
fitnesses = []
for delta in initial_delta:
    individual = incre_ps(delta, data)
    fitness = fitness_function(individual, true_effect)
    fitnesses.append(fitness)
fitnesses = np.array(fitnesses)

### Selection


In [184]:
def select_mating_pool(number_parents, population, fitnesses):
    """
    Select number of individual with the highest fitness
    """
    
    
    indices = (-fitnesses).argsort()[:number_parents]
    parents = population[indices, :]
    return parents

In [187]:
selected_individuals = select_mating_pool(300, initial_delta, fitnesses)

In [188]:
selected_individuals.shape

### Crossover


In [204]:
def cross_over(parents, number_of_offspring, crossover_point):
    new_population = []
    
    for i in parents:
        new_population.append(i)
    
    offspring = np.zeros((number_of_offspring, 672))
    for i in range(number_of_offspring):
        parent1_idx = i % parents.shape[0]
        parent2_idx = (i+1)% parents.shape[0]
        offspring[i, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[i, crossover_point:] = parents[parent2_idx, crossover_point:]
        new_population.append(offspring[i])
    new_population = np.array(new_population)
    return new_population

In [205]:
crossed_individuals = cross_over(selected_individuals, 100, 5)
crossed_individuals.shape

### Mutation

In [206]:
def mutation(new_population, mutation = 1):
    mutations_counter = 4
    for idx in range(new_population.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(mutation):
            random_value = np.random.uniform(-1.0, 1.0, 1)
            new_population[idx, gene_idx] = new_population[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return new_population

In [192]:
mutation(crossed_individuals, 2)

array([[0.02163649, 0.41709053, 0.38759095, ..., 0.50348067, 0.56017379,
        0.46745206],
       [0.16155893, 0.10382068, 0.71079089, ..., 0.47409612, 0.11246884,
        0.36487311],
       [0.3956946 , 0.31068936, 0.36501449, ..., 0.66218478, 0.7951693 ,
        0.82734737],
       ...,
       [0.76924952, 0.62774442, 0.10211664, ..., 0.10015515, 0.35607988,
        0.67855872],
       [0.16168693, 0.97146778, 0.85237895, ..., 0.29754531, 0.92246113,
        0.75778193],
       [0.28043303, 0.63806029, 0.95452839, ..., 0.92652488, 0.05667425,
        0.66161083]])

In [214]:
### fully programming
m1, m2 = 10000, 10000
a = np.random.rand(m1,data.shape[0])
# b = np.random.randint(low = 0, high=100, size=[m2,data.shape[0]])
initial_delta = np.array([a,b]).reshape((m1 + m2, data.shape[0]))

initial_delta = np.random.normal(50, 100, (10000,data.shape[0]))



number_of_generation = 4000
for i in tqdm(range(number_of_generation)):
    individuals = []
    fitnesses = []
    mae = []
    for delta in initial_delta:
        individual = incre_ps(delta, data)
        fitness = fitness_function(individual, true_effect)
        fitnesses.append(fitness)
        mae.append(np.mean(utils.abs_ate(individual, true_effect)))
    
    print("Delta {}".format(initial_delta[np.argmin(mae, axis=0)]))
    print("Generation {}. MAE {:.4f}".format(i, min(mae)))
    fitnesses = np.array(fitnesses)
    selected_individuals = select_mating_pool(5000, initial_delta, fitnesses)
    
    number_of_offspring = initial_population.shape[0] - selected_individuals
    
    print("Number of selected individuals {}".format(selected_individuals.shape))
    crossed_individuals = cross_over(selected_individuals, number_of_offspring, 5)
    print("Number of crossed individuals {}".format(crossed_individuals.shape))
    initial_delta = mutation(crossed_individuals, 2)  
    print("Number of mutated individuals {}".format(initial_delta.shape))
    print("\n")
    

  0%|          | 0/4000 [00:21<?, ?it/s]

Delta [  57.23956449  141.30936606  -11.08463045  107.98597612   15.74858385
  233.26481807    3.82821719   87.86111005  144.11097709  232.08846546
   58.26774022  150.54181873   47.06793608  -19.20697981  -35.7736785
  -75.20597203  150.06343887   -8.20634594   32.4385488    18.34709729
   95.42798426   47.44837237   -7.27754578   54.90606598 -180.98282809
    7.8573297    90.50896769  222.40828552  110.16190544  222.01395063
  -41.13301755   93.24354849  -71.84827207  -48.55910364   29.11759921
   37.43560053   69.32095812   24.51802602  -64.00103178  147.4551809
  -15.96248628   75.93573831  -68.96986285 -144.19889484   93.98654668
  -99.33473061  146.42953531  112.98603435  244.18231905  -43.10014568
   32.76676187   -6.59659284  189.88503787   97.6834094   131.15025428
  110.71763846  -27.28637672  127.88540728  117.18068539  211.95299317
  -84.68895189  -10.29458456   45.19496731   68.49823361   61.97464751
  191.96077873   45.36208284  -99.36486098  -40.82507088  250.19477593
  

NameError: name 'initial_population' is not defined

In [195]:
for delta in initial_delta:
    individual = incre_ps(delta, data)
    mae = np.mean(utils.abs_ate(individual, true_effect))
    print("MAE {:.4f}".format(mae))

MAE 10.8392
MAE 10.8329
MAE 10.8415
MAE 10.8465
MAE 10.8447
MAE 10.8555
MAE 10.8501
MAE 10.8496
MAE 10.8577
MAE 10.8586
MAE 10.8625
MAE 10.8600
MAE 10.8643
MAE 10.9111
MAE 10.8729
MAE 10.8730
MAE 10.8733
MAE 10.8756
MAE 10.8722
MAE 10.8805
MAE 10.8766
MAE 10.8765
MAE 10.8787
MAE 10.8812
MAE 10.8797
MAE 10.8785
MAE 10.8824
MAE 10.8819
MAE 10.8814
MAE 10.8814
MAE 10.8819
MAE 10.8816
MAE 10.8832
MAE 10.8831
MAE 10.8836
MAE 10.8689
MAE 10.9929
MAE 10.8844
MAE 10.8845
MAE 10.8884
MAE 10.8881
MAE 10.8877
MAE 10.8873
MAE 10.8878
MAE 10.8888
MAE 10.8886
MAE 10.8891
MAE 10.8895
MAE 10.8905
MAE 10.8909
MAE 10.8909
MAE 10.8916
MAE 10.8919
MAE 10.8927
MAE 10.8932
MAE 10.8928
MAE 10.8931
MAE 10.8940
MAE 10.8903
MAE 10.8969
MAE 10.8944
MAE 10.9160
MAE 10.8944
MAE 10.8962
MAE 10.8954
MAE 10.8944
MAE 10.8950
MAE 10.8959
MAE 10.8954
MAE 10.8967
MAE 10.8962
MAE 10.8926
MAE 10.8983
MAE 10.8963
MAE 10.9007
MAE 10.8998
MAE 10.8990
MAE 10.8988
MAE 10.9009
MAE 10.9003
MAE 10.8996
MAE 10.8887
MAE 10.8980
MAE 

In [3]:
import numpy as np
np.random.normal(0, 0.9, (3000,1))

array([[-1.25904245],
       [ 1.55772193],
       [ 1.44380754],
       ...,
       [-0.51596899],
       [ 0.38491367],
       [ 0.55890001]])